In [1]:
import torch

from data.ImageCaptionDataset import ImageCaptionDataset
from model.ImageCaptioningModel import ImageCaptioningModel
from data.vocab import Vocab

import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load vocab
vocab = Vocab("../data/train_data_preprocessed.pkl")

# Load model
model = ImageCaptioningModel(vocab_size=vocab.vocab_size, image_feature_size=2048, hidden_size=256, vocab=vocab)
model.load_state_dict(torch.load("../best_model.pth", map_location=device))
model.to(device)
model.eval()
from torchvision import transforms
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


KeyboardInterrupt: 

In [5]:
import pickle as pkl
data = None
with open("../data/test_data_preprocessed.pkl", "rb") as f:
    data = pkl.load(f)
    print(data[:5])

[{'id': 0, 'image_id': 1, 'caption': 'đây là khung cảnh xuất hiện ở phía trước một căn nhà', 'segment_caption': 'đây là khung_cảnh xuất_hiện ở phía trước một căn nhà', 'image_path': 'C:\\Users\\NguyenPC\\Desktop\\python_prj\\data/ktvic_dataset/public-test-images\\00000000001.jpg'}, {'id': 1, 'image_id': 1, 'caption': 'có một căn nhà cao tầng xuất hiện ở trong bức ảnh', 'segment_caption': 'có một căn nhà cao_tầng xuất_hiện ở trong bức ảnh', 'image_path': 'C:\\Users\\NguyenPC\\Desktop\\python_prj\\data/ktvic_dataset/public-test-images\\00000000001.jpg'}, {'id': 2, 'image_id': 1, 'caption': 'ở trong bức ảnh có sự xuất hiện của một căn nhà cao tầng', 'segment_caption': 'ở trong bức ảnh có sự xuất_hiện của một căn nhà cao_tầng', 'image_path': 'C:\\Users\\NguyenPC\\Desktop\\python_prj\\data/ktvic_dataset/public-test-images\\00000000001.jpg'}, {'id': 3, 'image_id': 1, 'caption': 'có một chiếc xe máy xuất hiện ở trong căn nhà', 'segment_caption': 'có một chiếc xe_máy xuất_hiện ở trong căn nhà'

In [5]:
import torch
from PIL import Image
import torch.nn.functional as F
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image.show()
    return transform(image).unsqueeze(0).to(device)

def generate_caption(image_path, max_length=29):
    image = preprocess_image(image_path)
    top_k = 5
    # Khởi tạo với token "<START>"
    caption = []
    input_seq = torch.tensor([[vocab.w2i["<START>"]]], dtype=torch.long).to(device)

    with torch.no_grad():
        for _ in range(max_length):

            output = model(image, input_seq)  # Dự đoán từ tiếp theo
            logits = output[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            top_k_probs, top_k_words = torch.topk(probs, top_k, dim=-1)
            sampled_id = torch.multinomial(top_k_probs, 1).item()  # Lấy 1 từ ngẫu nhiên trong top-k
            predicted_word = vocab.i2w[top_k_words[0][sampled_id].item()]

            if predicted_word == "<END>":  # Dừng khi gặp token "<END>"
                break

            caption.append(predicted_word)
            input_seq = torch.cat([input_seq, torch.tensor([[top_k_words[0][sampled_id].item()]], dtype=torch.long).to(device)], dim=1)

    return " ".join(caption)  # Bỏ token "<START>"

# Test với một ảnh
image_path = "img_1.png"
captions = [item["segment_caption"] for item in data if image_path in item['image_path']]
print("label:", captions)

print("predicted:", generate_caption(image_path, max_length=29))


label: []
predicted: <START> xung_quanh xung_quanh xung_quanh đối_diện đối_diện đối_diện đối_diện đối_diện đối_diện đối_diện thấy thấy thấy thấy thấy thấy thấy thấy thấy thấy thấy thấy đối_diện trước trước thấy đối_diện đối_diện
